In [4]:
import os
os.chdir('/Users/adityapatel/personal/projects/git/clash')

In [5]:
import pymysql
import pymysql.cursors
import config
import pandas as pd
import seaborn as sns

In [6]:
import numpy as np

In [7]:
import sqlalchemy

In [8]:
sns.set()

In [30]:
connection = pymysql.connect(host=config._mysql_host,
                             user=config._mysql_user,
                             password=config._mysql_password,
                             db=config._mysql_db,
                             port=config._mysql_port,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

In [31]:
# engine = sqlalchemy.create_engine(config.MYSQL_URL)

In [48]:
sql = '''
select * from player_stats_historic
'''

In [49]:
with connection.cursor() as cursor:
    # Create a new record
    cursor.execute(sql)
    results = cursor.fetchall()
    

In [51]:
df_raw = pd.DataFrame(results)

In [52]:
df = df_raw.copy(deep = True)

In [55]:
for_aiur = df[df['clan_tag'] == '#YUPCJJCR']

In [92]:
max_time = for_aiur.loc[for_aiur.groupby('player_name')['created_time'].idxmax()].set_index('player_tag')

In [93]:
min_time = for_aiur.loc[for_aiur.groupby('player_name')['created_time'].idxmin()].set_index('player_tag')

In [94]:
COLUMN_NAME = 'current_trophies'

In [98]:
max_col_name = '{}_max'.format(COLUMN_NAME)
min_col_name = '{}_min'.format(COLUMN_NAME)

diff_time_df = max_time[['player_name', COLUMN_NAME]].join(min_time[[COLUMN_NAME]], lsuffix='_max', rsuffix='_min')\
                                        .rename(columns={max_col_name: 'max', min_col_name:'min' })

In [99]:
diff_time_df['diff'] = diff_time_df['max'] - diff_time_df['min']

In [104]:
diff_time_df.sort_values('diff', ascending=False, inplace=True)

In [106]:
def get_improvement(df, player_tag, metric):
    max_col_name = '{}_max'.format(metric)
    min_col_name = '{}_min'.format(metric)
    
    for_aiur = df[df['clan_tag'] == '#YUPCJJCR']
    
    max_time = for_aiur.loc[for_aiur.groupby('player_name')['created_time'].idxmax()].set_index('player_tag')
    min_time = for_aiur.loc[for_aiur.groupby('player_name')['created_time'].idxmin()].set_index('player_tag')
    
    diff_time_df = max_time[['player_name', metric]].join(min_time[[metric]], lsuffix='_max', rsuffix='_min')\
                                        .rename(columns={max_col_name: 'max', min_col_name:'min' })
    
    diff_time_df['diff'] = diff_time_df['max'] - diff_time_df['min']
    
    diff_time_df.sort_values('diff', ascending=False, inplace=True)
    
    return diff_time_df

In [268]:
# default_return_cols = ['player_name', 'player_tag', 'war_stars', 'current_trophies']
class PlayerStats:
    def __init__(self, full_hist_df):
        self.full_hist_df = full_hist_df.copy(deep=True)
        self.player_df = None

        self.full_hist_df['player_name'] = self.full_hist_df['player_name'].str.lower()
        
    def set_player_tag(self, player_tag):
        self.player_df = self.full_hist_df[self.full_hist_df['player_tag'] == player_tag]
        
    def get_clan_df(self, clan_tag):
        self.clan_df =  self.full_hist_df[self.full_hist_df['clan_tag'] == clan_tag]
        
    def get_improvement_message(self, metric, player_tag=None, player_name=None):
        
        player_name = player_name.lower()
        
        assert player_tag or player_name, "Please input either player_tag or player_name"
        
        if player_name and not player_tag:
            player_df = self.full_hist_df[self.full_hist_df['player_name'] == player_name]
            
#             return player_df
            
            player_tag_list = player_df['player_tag'].unique()
            
            if player_tag_list.shape[0] > 1:
                raise ValueError("Duplicate Player Name. Please enter a player tag")
            else:
                player_tag = player_tag_list[0]
        else:
            player_df = self.full_hist_df[self.full_hist_df['player_tag'] == player_tag]
        
        max_metric = player_df.loc[player_df['created_time'] == player_df['created_time'].max()][metric]
        min_metric = player_df.loc[player_df['created_time'] == player_df['created_time'].min()][metric]
        
        diff = int(max_metric) - int(min_metric)
        
        message = '{player_name} changed {diff} in {metric}'.format(player_name=player_name.title(),
                                                                    diff=diff,
                                                                    metric=metric)
        
        return message       

In [284]:
PlayerStats(df).get_improvement_message('current_trophies', player_name='moo')

'Moo changed -21 in current_trophies'